In [1]:
import pandas as pd
from datetime import datetime

# Read data from the Excel file
meter_list = pd.read_excel('gorilla_test_data.xlsx', sheet_name='meter_list')
forecast_data = pd.read_excel('gorilla_test_data.xlsx', sheet_name='forecast_table')
rate_data = pd.read_excel('gorilla_test_data.xlsx', sheet_name='rate_table')

# Convert dtype to datetime64 and coerce errors
forecast_data['date'] = pd.to_datetime(forecast_data['date'], errors='coerce', infer_datetime_format=True)
rate_data['date'] = pd.to_datetime(rate_data['date'], errors='coerce', infer_datetime_format=True)
forecast_data.head()
rate_data.head()

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,NaN,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524


In [2]:
# Meter_list functions 

def return_aq_kwh(df_list, meter_id): 
    if meter_id not in df_list['meter_id'].values:
        raise ValueError(f"Meter ID {meter_id} not found in the meter_list.")
    
    aq_kwh = df_list.loc[df_list['meter_id'] == meter_id, 'aq_kwh'].values[0]
    print(f"The meter with ID {meter_id} corresponds to this aq_kwh: {aq_kwh}")
    return aq_kwh

def return_exit_zone(df_list, meter_id): 
    if meter_id not in df_list['meter_id'].values:
        raise ValueError(f"Meter ID {meter_id} not found in the meter_list.")
    
    exit_zone = df_list.loc[df_list['meter_id'] == meter_id, 'exit_zone'].values[0]
    print(f"The meter with ID {meter_id} corresponds to this exit zone: {exit_zone}")
    return exit_zone


# Forecast data functions 

# Convert input date in the correct datetime format
def convert_input_date(input_date):
    try:
        # Try to convert using the "DD/MM/YYYY" format
        converted_input_date = pd.to_datetime(input_date, format='%d/%m/%Y')
        return converted_input_date
    except ValueError:
        try:
            # If the first attempt fails, try the "YYYY-MM-DD" format
            converted_input_date = pd.to_datetime(input_date, format='%Y-%m-%d')
            return converted_input_date
        except ValueError:
            # If both attempts fail, raise an error
            raise ValueError("Invalid date format. Please provide a valid date.")

def return_kwh(df_forecast, meter_id, input_date): 
    converted_input_date = convert_input_date(input_date)
    mask = (df_forecast['meter_id'] == meter_id) & (df_forecast['date'] == converted_input_date)
    kwh = df_forecast.loc[mask, 'kwh'].values[0]
    print(f'The kwh usage of {meter_id} on {converted_input_date.date()} is: {kwh}')
    return kwh


# Rate data functions 

# Function to find the closest past date
def find_closest_past_date(df_rate, input_date):
    input_date = convert_input_date(input_date)

    # Filter only past dates
    past_dates = df_rate[rate_data['date'] <= input_date]['date']

    # If there are past dates, find the closest one
    if not past_dates.empty:
        closest_past_date = past_dates.iloc[(past_dates - input_date).abs().idxmin()]
        print(f"The date from which on the rates are applicable is: {closest_past_date.date()}")
        return closest_past_date
    else:
        return None
    


In [3]:
# Don't need functions anymore - use for testing 


# Create function to retrieve rate_p_per_kwh on date by meter_id and corresponding exit_zone. Return = array with 3 possible prices
def return_rate_by_date_meter_exit(df_list, meter_id, df_rate, input_date, exit_zone = None):
    converted_date = convert_input_date(input_date)
    rate_day = find_closest_past_date(df_rate, converted_date)
    if exit_zone == None: 
        exit_zone = return_exit_zone(df_list, meter_id)
    mask = (df_rate['exit_zone'] == exit_zone) & (df_rate['date'] == rate_day)
    rate_p_per_kwh = df_rate.loc[mask, ['aq_min_kwh', 'aq_max_kwh', 'rate_p_per_kwh']]
    return rate_p_per_kwh

# Daily charge function
def return_daily_charge_by_date_meter_exit(df_list, meter_id, df_forecast, df_rate, input_date):
    converted_date = convert_input_date(input_date)
    rate_p_per_kwh = return_rate_by_date_meter_exit(df_list, meter_id, df_rate, input_date)
    aq_kwh = return_aq_kwh(df_list, meter_id)
    daily_kwh = return_kwh(df_forecast, meter_id, input_date)

    if (aq_kwh < rate_p_per_kwh.iloc[0,1]) & (aq_kwh >= 0): 
        print(f"The rate per kwh for {converted_date.date()} is: {rate_p_per_kwh.iloc[0,2]} pennies")
        daily_charge_p = daily_kwh * rate_p_per_kwh.iloc[0,2]
    elif aq_kwh < rate_p_per_kwh.iloc[1,1]: 
        print(f"The rate per kwh for {converted_date.date()} is: {rate_p_per_kwh.iloc[1,2]} pennies")
        daily_charge_p = daily_kwh * rate_p_per_kwh.iloc[1,2]
    elif aq_kwh > rate_p_per_kwh.iloc[2,0]: 
        print(f"The rate per kwh for {converted_date.date()} is: {rate_p_per_kwh.iloc[2,2]} pennies")
        daily_charge_p = daily_kwh * rate_p_per_kwh.iloc[2,2]
    else: 
        raise ValueError
    daily_charge_pounds = daily_charge_p * 0.01
    print(f'The daily_charge is:\n{daily_charge_p.round(2)} pennies\n{daily_charge_pounds.round(4)} pounds')
    return daily_charge_p, daily_charge_pounds

In [4]:
# Table filters

# Date range filter
def daterange_filter(df_forecast, df_rate, start_date="01/04/2020", end_date="2030-12-31"): 
    start_date_converted = convert_input_date(start_date)
    end_date_converted = convert_input_date(end_date)

        # Check if start_date is earlier than the available data
    if (start_date_converted < df_rate['date'].min()):
        raise ValueError("Start date is earlier than the available data.")

    # Check if start_date is later than end_date
    if start_date_converted > end_date_converted:
        raise ValueError("Start date is later than end date.")
    
    # Check if start_date is later than the latest date in the forecast table
    if start_date_converted > df_forecast['date'].max():
        raise ValueError("Start date is later than the latest date available in the forecast table.")

    # Filter df_forecast based on the date range
    df_forecast_filtered = df_forecast[(df_forecast['date'] >= start_date_converted) & (df_forecast['date'] <= end_date_converted)]

    # Filter df_rate based on the date range
    closest_past_date_rate = find_closest_past_date(df_rate, start_date_converted)
    df_rate_filtered = df_rate[(df_rate['date'] >= closest_past_date_rate) & (df_rate['date'] <= end_date_converted)]
    return df_forecast_filtered, df_rate_filtered

# meter_id filter
def meter_id_filter(df_list, meter_id, df_forecast, df_rate): 
    exit_zone = return_exit_zone(df_list, meter_id)
    df_rate_filtered = df_rate[(df_rate['exit_zone'] == exit_zone)]
    df_forecast_filtered = df_forecast[(df_forecast['meter_id'] == meter_id)]
    return df_forecast_filtered, df_rate_filtered

# aq_kwh filter
def aq_kwh_filter(df_rate, aq_kwh):
    if (aq_kwh < df_rate.iloc[0, 3]) and (aq_kwh >= 0):
        # Keep 1st row, delete 2 rows, repeat
        df_rate_filtered = df_rate.iloc[::3]
    elif aq_kwh < df_rate.iloc[1, 3]:
        # Delete 1st row, keep 2nd row, delete 2 rows, repeat
        df_rate_filtered = df_rate.iloc[1::3]
    elif aq_kwh > df_rate.iloc[2, 2]:
        # Delete 1st and 2nd row, keep 3rd row, delete 2 rows, repeat
        df_rate_filtered = df_rate.iloc[2::3]
    else:
        raise ValueError("Invalid aq_kwh value.")
    return df_rate_filtered

def total_filter(df_list, meter_id, df_forecast, df_rate, start_date="01/04/2020", end_date="2030-12-31"): 
    print('\n--------------------------------------------\n')
    print(f'Meter id {meter_id} with Daterange: {start_date} until {end_date}')
    df_forecast_filtered, df_rate_filtered = daterange_filter(df_forecast, df_rate, start_date, end_date)
    df_forecast_filtered, df_rate_filtered = meter_id_filter(df_list, meter_id, df_forecast_filtered, df_rate_filtered)
    aq_kwh = return_aq_kwh(df_list, meter_id)
    df_rate_filtered = aq_kwh_filter(df_rate_filtered, aq_kwh)
    return df_forecast_filtered, df_rate_filtered

In [5]:
# Calculation of totals 

# Create new dataframe based on fc_filtered and rate_filtered
def results_meter_id(fc_filtered, rate_filtered): 
    df_result = pd.merge(fc_filtered, rate_filtered, how='left', on='date')
    df_result.drop(['aq_min_kwh', 'aq_max_kwh'], axis=1, inplace=True)
    df_result[['exit_zone', 'rate_p_per_kwh']] = df_result[['exit_zone', 'rate_p_per_kwh']].fillna(rate_filtered.iloc[0][['exit_zone', 'rate_p_per_kwh']])
    # Forward fill the missing values in rate_filtered columns since empty columns due to left join
    df_result[['exit_zone', 'rate_p_per_kwh']] = df_result[['exit_zone', 'rate_p_per_kwh']].ffill()
    return df_result

# Calculate daily charges in pounds and round
def calculate_daily_charge(df_results): 
    df_results['daily_charge_pounds'] = df_results['kwh'] * df_results['rate_p_per_kwh'] * 0.01
    df_results['daily_charge_pounds'] = df_results['daily_charge_pounds'].round(2)
    return df_results

# Aggregated by meter_id
def results_aggregated(df_results): 
    sum_results = df_results.groupby('meter_id')[['kwh', 'daily_charge_pounds']].sum()
    sum_results = sum_results.rename(columns={'kwh': 'total_kwh', 'daily_charge_pounds': 'total_charge_pounds'})
    return sum_results

def get_results(df_list, meter_id, df_forecast, df_rate, start_date="01/04/2020", end_date="2030-12-31"): 
    df_forecast_filtered, df_rate_filtered = total_filter(df_list, meter_id, df_forecast, df_rate, start_date, end_date)
    df_results = results_meter_id(df_forecast_filtered, df_rate_filtered)
    df_results = calculate_daily_charge(df_results)
    sum_results = results_aggregated(df_results)
    return sum_results

In [6]:
get_results(meter_list, 14676236, forecast_data, rate_data).head()


--------------------------------------------

Meter id 14676236 with Daterange: 01/04/2020 until 2030-12-31
The date from which on the rates are applicable is: 2020-04-01
The meter with ID 14676236 corresponds to this exit zone: EA1
The meter with ID 14676236 corresponds to this aq_kwh: 28978


,total_kwh,total_charge_pounds
meter_id,,
14676236,28978.0,77.16


In [7]:
# All meter_id results using a loop (not sustainable for bigger datasets!)
def get_results_for_multiple_ids(df_list, meter_ids, df_forecast, df_rate, start_date="01/04/2020", end_date="2030-12-31"):
    results_list = []
    for meter_id in meter_ids:
        results = get_results(df_list, meter_id, df_forecast, df_rate, start_date, end_date)
        results_list.append(results)
    all_results = pd.concat(results_list)
    return all_results.reset_index()

# Example usage with a list of meter IDs
meter_ids_list = [14676236, 34509937, 50264822]
results_df = get_results_for_multiple_ids(meter_list, meter_ids_list, forecast_data, rate_data)
results_df.head()



--------------------------------------------

Meter id 14676236 with Daterange: 01/04/2020 until 2030-12-31
The date from which on the rates are applicable is: 2020-04-01
The meter with ID 14676236 corresponds to this exit zone: EA1
The meter with ID 14676236 corresponds to this aq_kwh: 28978

--------------------------------------------

Meter id 34509937 with Daterange: 01/04/2020 until 2030-12-31
The date from which on the rates are applicable is: 2020-04-01
The meter with ID 34509937 corresponds to this exit zone: SO1
The meter with ID 34509937 corresponds to this aq_kwh: 78324

--------------------------------------------

Meter id 50264822 with Daterange: 01/04/2020 until 2030-12-31
The date from which on the rates are applicable is: 2020-04-01
The meter with ID 50264822 corresponds to this exit zone: NT1
The meter with ID 50264822 corresponds to this aq_kwh: 265667


,meter_id,total_kwh,total_charge_pounds
0,14676236,28978.0,77.16
1,34509937,78324.0,215.47
2,50264822,265667.0,581.12
